# 10. 얼굴인식(Haar Cascade)

In [1]:
import cv2 as cv
import numpy as np
SONNY = "../images/son.png"
FAMILY = "../images/family.jpg"
WOMEN = "../images/women.jpg"
MAN = "../images/man.jpg"

FACE_CASCADE = "../cascade/haarcascade_frontalface_default.xml"
EYE_CASCADE = "../cascade/haarcascade_eye.xml"

## 얼굴인식
- `cv2.detectMultiScale(image, scaleFactor, minNeighbors, flags, 
minSize, maxSize)`
    - OpenCV의 객체 검출 함수로, 주로 얼굴 검출과 같은 작업에 사용
    - `scaleFactor` : 이미지 크기를 얼마나 줄여가며 검출을 진행할지를 결정
    - `minNeighbors` 얼마나 많은 주변에 객체로 간주할지를 결정, 값이 크면 더 강한 검출 기준을 설정
    - `minSize` : 검출할 객체의 최소 크기

In [3]:
face_cascade = cv.CascadeClassifier(FACE_CASCADE)
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

img = cv.imread(SONNY)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=2, minSize=(10,10)
)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=1, maxSize=(10,10)
)
# print(eyes)
# print(faces)
if len(faces):
    for face in faces:
        x, y, width, height = face
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)

if len(eyes):
    for eye in eyes:
        x, y, width, height = eye
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,255), 2, cv.LINE_AA)

cv.imshow("img", img)
cv.waitKey(0)
cv.destroyAllWindows()


In [4]:
# 실습1. 캠화면에 적용
# 캠화면에서 얼굴과 눈 찾아서 얼굴 및 눈 영역에 사각형 표시
# 네모위에 "Face", "Eye"라는 글자 띄우기
face_cascade = cv.CascadeClassifier(FACE_CASCADE)
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(10,10))

    for (x, y, weight, height) in faces:
        cv.rectangle(frame, (x, y), (x+weight, y+height), (0, 0, 255), 2)
        cv.putText(frame, "Face", (x, y-10), cv.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 2)

        roi_gray = gray[y:y+height, x:x+weight]
        roi_color = frame[y:y+height, x:x+weight]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (eye_x, eye_y, eye_weight, eye_height) in eyes:
            cv.rectangle(roi_color, (eye_x, eye_y), (eye_x+eye_weight, eye_y+eye_height), (0, 255, 0), 2)
            cv.putText(roi_color, "Eye", (eye_x, eye_y-10), cv.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 2)

    cv.imshow("Face & Eye Detection", frame)

    if cv.waitKey(1) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

In [2]:
# 실습2. 귀여운 눈 덮어 씌우기
# 원하는 사람 이미지에서 눈을 찾아서
# 해당 영역에 귀여운 눈을 덮어 씌워보기
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)
img = cv.imread(WOMEN)
resized = cv.resize(img, None, fx=0.1, fy=0.1, interpolation= cv.INTER_NEAREST)
eye_icon = cv.imread("../images/eye.png")
gray = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=2, maxSize=(100,100)
)

if len(eyes):
    for eye in eyes:
        x, y, width, height = eye
        resized[y:y+height, x:x+width] = cv.resize(eye_icon, (width, height))

cv.imshow("img", resized)
cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
# 실습3
# 웹캠을 통해 들어오는 영상을 이용해 움직임이 감지될 경우
# 해당 프레임을 캡쳐해서 파일로 저장하는 프로그램 만들기
# 핵심
# - VideoCapture()
# - createBackgroundSubractorMOG2()
# - findContour()
cap = cv.VideoCapture(0)
# 배경 제거
bgs = cv.createBackgroundSubtractorMOG2()
count = 0 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    fgmask = bgs.apply(frame) # 배경 제거 적용
    contours, _ = cv.findContours(fgmask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv.contourArea(contour) < 5000: # 움직임 크기가 작은건 무시
            continue

        x, y, weight, height = cv.boundingRect(contour)
        cv.rectangle(frame, (x, y), (x+weight, y+height), (0, 255, 0), 2)

        save_path = "../output/"
        filename = f"{save_path}motion_{count}.jpg"
        cv.imwrite(filename, frame)
        print(f"움직임 감지 → {filename} 저장 완료!")
        count += 1   

    cv.imshow("Frame", frame)
    cv.imshow("FG Mask", fgmask)

    if cv.waitKey(1) == ord("q"):  # ESC
        break

cap.release()
cv.destroyAllWindows()


움직임 감지 → ../output/motion_0.jpg 저장 완료!
움직임 감지 → ../output/motion_1.jpg 저장 완료!
움직임 감지 → ../output/motion_2.jpg 저장 완료!


In [5]:
import cv2
import time
import os 

# 폴더 생성
SAVE_DIR = "captures"

# 최소영역
MIN_AREA = 1200

# 저장 간격(초단위)
COOLDOWN = 1.0

# 폴더가 없으면 생성
os.makedirs(SAVE_DIR, exist_ok=True)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("웹캠 오류")

# 배경 제거
# createBackgroundSubtractorMOG2
# 웹캠이나 cctv영상처럼 움직이는 물건(전경)과 고정된 배경을 자동으로 구분할때 사용:
# history() : 기본값 500, 값이 크면 배경 모델이 천천히 변함(안정적), 작으면 빠르게 변함(민감)
# varThreshold : 기본값 16, 픽셀이 전경인지 배경인지를 판단
    # 값이 작은 변화에도 움직임으로 판단
    # 값이 크면 큰 변화가 있어야 움직임으로 판단
# detectShadows : 기본값 True, 그림자도 검출할지 여부
backsub = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=25, detectShadows=True)

last_save = 0.0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1번
    # 전경 마스크 받아오기
    fg = backsub.apply(frame) # 현재 프레임에서 움직임 부분만 추출

    # 그림자 제거 : 200이상 부분만 남기고 나머지는 0으로 처리
    _, fg = cv2.threshold(fg, 200, 255, cv2.THRESH_BINARY)

    # 움직임 영역을 키워서 구멍 메우기
    fg = cv2.dilate(fg, None, iterations=2)

    # 2번
    # 움직임 영역 찾기
    contours, _ = cv2.findContours(fg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion = False # 움직임이 있었는지 표시하는 변수

    for c in contours:
        # 너무 작은 영역은 무시
        if cv2.contourArea(c) < MIN_AREA:
            continue

        # 움직임이 있는 부분에 사각형 표시
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame, (x,y), (x + w, y + h), (0,255,0), 2)
        motion = True

    # 화면 상단에 "Motion: ON/OFF" 글자 표시
    status_text = "ON" if motion else "OFF"
    cv2.putText(frame, f"Motion: {status_text}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0) if motion else(0,0,255), 2)
        
    # 원본 영상과 마스크 영상 출력
    cv2.imshow('frame', frame)
    cv2.imshow('mask', fg)

    # 3번 움직인 사진 저장
    now = time.time()
    if motion and (now - last_save > COOLDOWN):
        filename = time.strftime("%Y%m%d_%H%M%S") + ".jpg" # 현재 시각으로 파일명 생성
        cv2.imwrite(os.path.join(SAVE_DIR, filename), frame) # 현재 프레임 저장
        last_save = now
        
    # 4번 종료
    if cv2.waitKey(1) == ord("q"):
        break

# 카메라, 창 종
cap.release()
cv2.destroyAllWindows()